In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## **Step 1: Create empty folders for all unique car models based on SGCarMart dataset**

In [ ]:
#Import data from carmodel csv. It contains all the listing ID and information uploaded by SGCarMart's past users

data='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/CarModel.csv'

rawcarmodel = pd.read_csv(data)

rawcarmodel

In [ ]:
#split column 'carmodel' to get the car model and car make name only
rawcarmodel['Car Model1'] = rawcarmodel.carmodel.str.split().str.get(0)
rawcarmodel['Car Model2'] = rawcarmodel.carmodel.str.split().str.get(1)

#create new column 'car model' to combine the car model and car make 
rawcarmodel['Car Model'] = rawcarmodel['Car Model1'] + ' ' + rawcarmodel['Car Model2']
rawcarmodel.drop(rawcarmodel.columns.difference(['aid','Car Model']), 1, inplace=True)
rawcarmodel

In [ ]:
#create image folders for all unique car models if not exists
a = rawcarmodel['Car Model'].unique()

path = "/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/"

for x in range(len(a)):
  if not os.path.exists(path + str(a[x])):
    os.mkdir(path + str(a[x]))

In [ ]:
#delete folders for all unique car models (In case need to recreate everything)

for x in range(len(a)):
  os.rmdir(path + str(a[x]))

In [ ]:
#define the folder path we created for all classes of car models
saved_folder = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/'


#count number of classes in the datasets
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(saved_folder):

    for directories in dirs:
        totalDir += 1

    for Files in files:
        numfiles +=1
        totalFiles += 1

#numbe of directories = number of classes
print('Total Number of directories',totalDir)


## **Step 2: Inspect SGCarMart Dataset Size (Number of total images)**

In [ ]:
#count number files in folders
import os, os.path

#the initial images provided by SGCarmart are split into 7 folders due to large file size 
folder1='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images/'
folder2='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images2/'
folder3='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images3/'
folder4='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images4/'
split1='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images4_1/'
split2='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images4_2/'
split3='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images4_3/'


#print the number of images in each folder
print(len([name for name in os.listdir(folder1) if os.path.isfile(os.path.join(folder1, name))]))
print(len([name for name in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, name))]))
print(len([name for name in os.listdir(folder3) if os.path.isfile(os.path.join(folder3, name))]))
print(len([name for name in os.listdir(folder4) if os.path.isfile(os.path.join(folder4, name))]))
print(len([name for name in os.listdir(split1) if os.path.isfile(os.path.join(split1, name))]))
print(len([name for name in os.listdir(split2) if os.path.isfile(os.path.join(split2, name))]))
print(len([name for name in os.listdir(split3) if os.path.isfile(os.path.join(split3, name))]))



## **Step 3: Detect car in raw images, crop raw images and distribute the cropped raw images to their respective class folders created in Step 1**

This section will detect the existence of car in the images and return the coordinates of bounding box.

In [ ]:
sys.path.append('/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/')
print("All the system paths:")
sys.path

In [ ]:
os.chdir('/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/')
print("Current working directory:")
os.getcwd()

In [ ]:
def cv2plt(img):
    plt.figure(figsize=(8,8))        # To change the size of figure
    plt.axis('off')
    if np.size(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img,cmap='gray',vmin=0,vmax=255)  
    plt.show()

print(cv2plt)

In [ ]:
lbl_file        = 'yolov3.txt'
classes         = open(lbl_file).read().strip().split("\n")

                                                # Read in the deep learning net
yoloconfig      = 'yolov3.cfg'
yoloweights     = 'yolov3.weights'
net             = cv2.dnn.readNet(yoloweights,yoloconfig)

pd.DataFrame(classes)                           # Display the first five and the last five classes

In [ ]:
i=0

def yoloV3Detect(img,scFactor=1/255,nrMean=(0,0,0),RBSwap=True,scoreThres=0.5,nmsThres=0.4):

  blob = cv2.dnn.blobFromImage(image=img,scalefactor=1/255,size=(416, 416),mean=(0,0,0),swapRB=True,crop=False)

  imgHeight = img.shape[0]
  imgWidth = img.shape[1]

  net.setInput(blob)
  outLyrs = getOutputLayers(net)
  preds = net.forward(outLyrs)


  classId = []
  confidences=[]
  boxes=[]




  for scale in preds:
    for pred in scale:
      scores = pred[5:]
      clss = np.argmax(scores)
      confidence = scores[clss]

      if confidence > 0.5:
        xc = int(pred[0]*imgWidth) 
        yc = int(pred[1]*imgHeight) 
        w = int(pred[2]*imgWidth) 
        h = int(pred[3]*imgHeight)
        x = xc - w/2
        y = yc - h/2

        classId.append(clss) 
        confidences.append(float(confidence)) 
        boxes.append([x, y, w, h])

  fclasses=classId
  fboxes=boxes
 

  return [fboxes,fclasses,confidences]



def getOutputLayers(net):
  layers = net.getLayerNames()
  outLayers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]

  return outLayers

folder1='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images/'
folder2='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images2/'
folder3='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images3/'
folder4='/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/images4/'



for filename in os.listdir(folder4):




  img=cv2.imread(folder4 + filename)

  #print(filename[:-7])
  cc=filename[:-7]
  x=yoloV3Detect(img,scFactor=1/255,nrMean=(0,0,0),RBSwap=True,scoreThres=0.5,nmsThres=0.4)

  #print(x)


#find bounding box
  scoreThres = 0.5
  nmsThres = 0.4
  selected = cv2.dnn.NMSBoxes(bboxes=x[0], scores=x[2],score_threshold=scoreThres, nms_threshold=nmsThres)


  colorset = np.random.uniform(0, 255,size=(len(classes),3))

  box22=x[0]
  class22=x[1]

  #print(selected)
  if len(selected) !=0:
    for j in selected[:,0]:
      box     = box22[j]
      color   = colorset[class22[j]]
      txtlbl  = str(classes[class22[j]])
      x       = int(box[0])
      y       = int(box[1])
      w       = int(box[2])
      h       = int(box[3])

      if txtlbl == 'car':
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        #print(txtlbl)
  
        cv2.putText(img, txtlbl,(x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,color,1,cv2.LINE_AA)

#crop image
        cropped_image = img[y:y+h, x:x+w]

        saved_folder = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/'

        rowfound = rawcarmodel[rawcarmodel['aid']==int(cc)].index.values
        try:
          carmodelfound= rawcarmodel.loc[rowfound[0], 'Car Model']
          #print(cc)
          #print(rowfound)
          #print(carmodelfound)


        
          #cv2plt(cropped_image)

          cv2.imwrite(saved_folder + carmodelfound + '/'+ cc + '_____' + str(i) + '.jpg', cropped_image)    
      
        except:
          print("Something else went wrong")


        i=i+1


The cell below deletes photos in each classes if photo size is below 50kb. This is to ensure that we have high quality data and not partial car image exists in the dataset. The algorithm also helps to delete classes that have no images exists in folder.

In [ ]:
#delete photos in folders if photo size <50kb.

import os
rootdir = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filesize = os.path.getsize(subdir + '/' + file)
        if int(filesize) < 50000:
            os.remove(subdir + '/' + file)
            print('Removed' + subdir + '/'+ file)

#delete folder if contains 0 photo
for dirpath, dirnames, files in os.walk(rootdir):
    if files:
        print(dirpath, 'has files')
    if not files:
        #os.rmdir(dirpath)
        print(dirpath, 'is empty')

## **Step 4: Create train and test sets by splitting cropped images in folders created in Step 1**

Split the data into train and test set for each class, the folder structure will look like this:
  
  
    |Car Models

    |____Train
      
      |____ Volvo XC90
            
            |____image_1.png

            |____image_2.png

            |____ .......


    |____Test
      
      |____ Volvo XC90
            
            |____image_3.png

            |____image_4.png

            |____ .......




In [ ]:
import os, shutil


saved_folder = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/'


#count number of files in each classes
totalFiles = 0
totalDir = 0

numoffiles =[]
carmodel=[]
dir=[]

for base, dirs, files in os.walk(saved_folder):


    for directories in dirs:
        totalDir += 1
        #print(directories)
        dir.append(directories)
    numfiles=0
    for Files in files:
        numfiles +=1
        totalFiles += 1

    numoffiles.append(numfiles)
    carmodel.append(base)

In [ ]:
#create folders for train and test


# Path to the directory where the original dataset was uncompressed
original_dataset_dir = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/'

# Directory where you’ll store your smaller dataset
base_dir = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset_split/'

#os.mkdir(base_dir)


#create directory for train & test folder for each class
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


for i in range(totalDir):
  train1=os.path.join(base_dir,'train', dir[i])
  test1=os.path.join(base_dir,'test', dir[i])

  os.mkdir(train1)
  os.mkdir(test1)




In [ ]:
#split dataset

import os, shutil

saved_folder = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/'


#count number of files in each classes
totalFiles = 0
totalDir = 0

numoffiles =[]
carmodel=[]
dir=[]

for base, dirs, files in os.walk(saved_folder):


    for directories in dirs:
        totalDir += 1
        #print(directories)
        dir.append(directories)
    numfiles=0
    for Files in files:
        numfiles +=1
        totalFiles += 1

    numoffiles.append(numfiles)
    carmodel.append(base)

for i in range(totalDir):
  

  os.chdir('/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/' + dir[i])
  src_dir='/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset/' + dir[i]
  train_dst_dir = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset_split/train/' + dir[i]
  test_dst_dir = '/content/gdrive/My Drive/ISY5002/Project/Image_Search/model_dataset_split/test/' + dir[i]

#check if the the training directory is empty for the class
  for x,y,z in os.walk(train_dst_dir):
    if len(z)==0:

      print(train_dst_dir + ' is empty.')
      fnames=[]
      for x,y,z in os.walk(src_dir):

          fnames=z
  #print(src_dir)
      #print(fnames)
      total_num_of_file=len(fnames)

      if total_num_of_file!=0:
        xx=round(total_num_of_file - total_num_of_file/4)
        fnames_train=fnames[0:xx]
        fnames_test=fnames[xx:]

        for fname1 in fnames_train:
            src = os.path.join(src_dir, fname1)
            print(src)
            dst = os.path.join(train_dst_dir, fname1)
            print(dst)
            shutil.copyfile(src, dst)
        print('finish copying training set' + dir[i] )

        for fname2 in fnames_test:
            src = os.path.join(src_dir, fname2)
            dst = os.path.join(test_dst_dir, fname2)
            shutil.copyfile(src, dst)
        print('finish copying testing set' + dir[i] )


    else:
        print(train_dst_dir + ' is not empty.')
